In [1]:
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from keras.layers.normalization import (BatchNormalization)
from keras.models import Sequential, load_model
import sys
import numpy as np
import os
import glob
from PIL import Image
from datetime import datetime



Using TensorFlow backend.


In [43]:
def c3d(num_classes, shape):
    
    
    model = Sequential()
    # conv_1
    conv_1 = Conv3D(32, (7,7,7), input_shape=shape)
    model.add(conv_1)
    print(conv_1.output_shape)
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    
    # conv_2 
    conv_2 = Conv3D(64, (3,3,3))
    model.add(conv_2)
    print(conv_2.output_shape)
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
    
    # conv_3
    conv_3 = Conv3D(128, (2,2,2))
    model.add(conv_3)
    print(conv_3.output_shape)
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))


    #
    model.add(Flatten())
    d1 = Dense(256)
    model.add(d1)
    print (d1.output_shape)
    model.add(Dropout(0.2))
    
    d2 = Dense(256)
    model.add(d2)
    print (d2.output_shape)
    model.add(Dropout(0.2))
    
    d3 = Dense(num_classes, activation='softmax')
    model.add(d3)
    print (d3.output_shape)
    return model

In [3]:
import glob
from PIL import Image

tic = datetime.now()
def dataPreProcess(num_videos, frame_dir, shape = (112, 112, 3), num_frames = 10):
    h, w, c = shape
    print (h,w,c)
    data = np.zeros((num_videos, num_frames, h, w, c))
    
    for i in range(num_videos):
        path = frame_dir +'/video'+str(i)
        if not os.path.exists(path): continue
        
        count = 0
        for fn in glob.glob(path+'/*.jpg'):
            im = Image.open(fn)
            im_resized = im.resize((h, w),Image.ANTIALIAS)

            # transform to array
            im_arr = np.asarray( im_resized, dtype="int32" )
            im_arr = np.expand_dims(im_arr, axis=0)
            
            im_arr.flags.writeable = True
            im_arr = im_arr.astype(np.float64)
            ## im_arr = preprocess_input(im_arr)   substraction, normaliztion ....
            
            data[i, count] = im_arr
            count += 1
        if i%20 == 0:
            print(datetime.now()-tic, i)
    return data
toc = datetime.now()


In [42]:
# parameters setting
from keras.utils import to_categorical
    
curr_path = os.getcwd()
frame_dir = curr_path + '/datasets/frames'
num_videos = 500
num_frames = 10
w, h, c = 64, 64, 3

y = np.load(curr_path+'/datasets/category.npy')
y = np.asarray(y[:num_videos])[:,1]
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes=num_classes)

x = dataPreProcess(num_videos, frame_dir, (w,h,c))


64 64 3
0:34:22.699764 0
0:34:24.123636 20
0:34:25.503964 40
0:34:26.942209 60
0:34:28.179587 80
0:34:29.482137 100
0:34:30.907742 120
0:34:32.168994 140
0:34:33.281320 160
0:34:34.820065 180
0:34:36.166035 200
0:34:37.548726 220
0:34:39.029104 240
0:34:40.493240 260
0:34:42.001201 280
0:34:43.557278 300
0:34:44.889395 320
0:34:46.022367 340
0:34:48.880568 380
0:34:50.447674 400
0:34:51.929578 420
0:34:53.250121 440
0:34:54.359399 460
0:34:55.826597 480


In [44]:
# train
print (num_classes)
model = c3d(num_classes, (num_frames, w, h, c))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print (x.shape)
print (y.shape)
model.fit(x, y, epochs=10, batch_size=32, validation_split = 0.2, verbose = 1)

20
(None, 4, 58, 58, 32)
(None, 2, 27, 27, 64)
(None, 1, 12, 12, 128)
(None, 256)
(None, 256)
(None, 20)
(500, 10, 64, 64, 3)
(500, 20)
Train on 400 samples, validate on 100 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[256,256]
	 [[Node: dense_49/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=5747007, _device="/job:localhost/replica:0/task:0/gpu:0"](dense_49/random_uniform/shape)]]

Caused by op 'dense_49/random_uniform/RandomUniform', defined at:
  File "/home/sunjiajun/anaconda3/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sunjiajun/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sunjiajun/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/sunjiajun/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/sunjiajun/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/ho